In [34]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from scipy.stats import f_oneway
import re
sns.set()

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import os

#from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from sklearn.metrics import f1_score, confusion_matrix, classification_report, cohen_kappa_score, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# Login to Hugging face

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
class TensorBoardLogger:
    def __init__(self, log_dir):
        self.summary_writer = SummaryWriter(log_dir)

    def add_scalars(self, tag_step_value_dict):
        """
        :param parent_tag: str, e.g. "Training Loss"
        :param tag_step_value_dict: dict, e.g., {"key":(step, value), "q_grad":(10000, 1.11)}
        """
        for tag, (step, value) in tag_step_value_dict.items():
            self.summary_writer.add_scalar(tag, value, step)

# Loading Essay Score Dataset & perfom NLP Data Preprocessing

In [4]:
data_path = "data/train.csv"
df = pd.read_csv(data_path)

data_path = "data/test.csv"
df_test = pd.read_csv(data_path)

In [28]:
X = df.drop(columns=["score","essay_id"])
y = df["score"] - 1
test = df_test.drop(columns=["essay_id"])


## NLP Preprocessing

In [29]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)


cList = {#"dont" : "do not", "doesnt" : "does not", "thats" : "that is"
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would",  ## --> he had or he would
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is", 
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   ## --> I had or I would
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",   ## --> It had or It would
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",   ## --> It had or It would
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
}
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    #x = re.sub("'\d+", '',x)
    #x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Remove \xa0
    x = x.replace(u'\xa0',' ')
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
#     x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
  #replace \'s with 's
    #print(re.findall("\\'s", x))
    #x = re.sub(r"\[\]'s", "'s", x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

In [30]:
X["full_text"] = X.apply(lambda x: dataPreprocessing(x["full_text"]),axis=1)
test["full_text"] = test.apply(lambda x: dataPreprocessing(x["full_text"]),axis=1)


In [8]:
len(X)

17307

## Create Train-Validation Pandas Dataset Split Note: Huggingface trainer needs the datasets to be compatible with huggingface dataset format

In [31]:

seed = 10
generator = np.random.RandomState(seed)
df_size = len(X)
train_proportion = 0.8
validation_proportion = 0.2
train_size = int(df_size * train_proportion)
validation_size = df_size - train_size 
arr_train_idxs = generator.choice(np.arange(1,len(X)),size=[train_size,],replace=False)

train = X.iloc[arr_train_idxs].reset_index(drop=True)
train_labels = y.iloc[arr_train_idxs].reset_index(drop=True)
validation = X[~X.index.isin(arr_train_idxs)].reset_index(drop=True)
validation_labels = y.iloc[~y.index.isin(arr_train_idxs)].reset_index(drop=True)

In [35]:
test.head()

,full_text
0,many people have car where they live. the thin...
1,i am a scientist at nasa that is discussing th...
2,people always wish they had the same technolog...


In [37]:
train_concat = pd.concat([train,train_labels],axis=1)
validation_concat =pd.concat([validation,validation_labels],axis=1)

In [39]:
# from pandas dataframe datasets to huggingface datasets objects...
dataset_train = Dataset.from_pandas(train_concat)
dataset_val = Dataset.from_pandas(validation_concat)
dataset_test = Dataset.from_pandas(test)

# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val
})
dataset

DatasetDict({
    train: Dataset({
        features: ['full_text', 'score'],
        num_rows: 13845
    })
    val: Dataset({
        features: ['full_text', 'score'],
        num_rows: 3462
    })
})

## Classes weights

In [10]:
class_weights = train_labels.value_counts(normalize=True).sort_index().to_list()
class_weights = torch.Tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

tensor([0.0716, 0.2764, 0.3627, 0.2243, 0.0567, 0.0084])

# Fine Tuning Llama-3 for Essay Score Assesment using PEFT technique: QLORA

## Importing model & tokenizer

In [11]:
from transformers import AutoModel
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

from accelerate import Accelerator

In [12]:
config = {
    'model': "meta-llama/Meta-Llama-3-8B",
    'dropout': 0.2,
    'max_length': 2048,
    'batch_size': 1, # anything more results in CUDA OOM [for unfreezed encoder] on Kaggle GPU
    'epochs': 10,
    'lr': 3e-4,
    'enable_scheduler': True,
    'scheduler': 'CosineAnnealingWarmRestarts',
    'gradient_accumulation_steps': 2,
    'adam_eps': 1e-6, # 1e-8 default
    'freeze_encoder': True
}

### Quantization Config (for QLORA)

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

### Lora Config

In [17]:
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    config["model"],
    quantization_config=quantization_config,
    num_labels=class_weights.shape[0]
)

/home/kevinmg96/miniconda3/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


prepare_model_for_kbit_training() function to preprocess the quantized model for training.

In [15]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


get_peft_model prepares a model for training with a PEFT method such as LoRA by wrapping the base model and PEFT configuration with get_peft_model

In [18]:
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

Load the tokenizer

In [19]:
tokenizer = AutoTokenizer.from_pretrained(config["model"], add_prefix_space=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

### Testing the tokenizer...

In [61]:
text = train["full_text"][0]
text

'the face on mars is a landformation. people would argue it isnt but keep in mind that isnt the case with this. with people argueing that it isnt they think aliens did it but that may not have happend. the face of mars has to be a landformation. the face on mars is a landformation because natural occurances happen. this could have happend fromm a meteor strike or the winds in mars. not to mention if the clouds in mars are constant then finding this twice on over a 20 year period should still be the same fro slow winds and mild weater. many people belive aliens were the cause of the face. well thats nearly impossible if you consider the aliens were made in mars. life forms take upon millions of years to grow and need about 60 degree weather on average and mars has below freezing average. plus if theree were life on mars, there should be green on mars because of plants. people also think its aliens because of the lattitude changed. well this is because the camera that had to take this pi

In [62]:
encoded =tokenizer.encode(text)
len(encoded)

310

In [63]:
decoded = tokenizer.decode(encoded) 
decoded

'<|begin_of_text|>the face on mars is a landformation. people would argue it isnt but keep in mind that isnt the case with this. with people argueing that it isnt they think aliens did it but that may not have happend. the face of mars has to be a landformation. the face on mars is a landformation because natural occurances happen. this could have happend fromm a meteor strike or the winds in mars. not to mention if the clouds in mars are constant then finding this twice on over a 20 year period should still be the same fro slow winds and mild weater. many people belive aliens were the cause of the face. well thats nearly impossible if you consider the aliens were made in mars. life forms take upon millions of years to grow and need about 60 degree weather on average and mars has below freezing average. plus if theree were life on mars, there should be green on mars because of plants. people also think its aliens because of the lattitude changed. well this is because the camera that ha

## Pytorch Datasets & Dataloaders

In [21]:
class EssayScoringDataset:
    def __init__(self, df,y, config, tokenizer=None, is_test=False):
        self.df = df
        self.y = y
        self.max_len = config["max_length"]
        self.tokenizer = tokenizer
        self.bos_token_id = self.tokenizer.bos_token_id
        self.eos_token_id = self.tokenizer.eos_token_id
        self.is_test = is_test
        
    def __getitem__(self,idx):
        essay = self.df.iloc[idx]["full_text"]
        if self.y is not None:
            score = self.y[idx]


        tokenized = tokenizer.encode_plus(essay,
                                          None,
                                          add_special_tokens=True,
                                          max_length=self.max_len,
                                          truncation=True,
                                          padding='max_length'
                                         )
        

        inputs = {
            "input_ids": torch.tensor(tokenized['input_ids'], dtype=torch.long),
            "attention_mask": torch.tensor(tokenized['attention_mask'], dtype=torch.long)
        }
        
        if self.is_test == True:
            return inputs
        
        targets = {
            "labels": torch.tensor(score).type(torch.LongTensor),
        }
        
        return inputs, targets
    
    def __len__(self):
        return len(self.df)

In [22]:
train_pytorch_dataset = EssayScoringDataset(train,train_labels,config,tokenizer)
validation_pytorch_dataset = EssayScoringDataset(validation,validation_labels,config,tokenizer)
test_pytorch_dataset = EssayScoringDataset(validation,None,config,tokenizer,True)


train_loader = torch.utils.data.DataLoader(train_pytorch_dataset, 
                                           batch_size=config["batch_size"])
valid_loader = torch.utils.data.DataLoader(validation_pytorch_dataset,batch_size=config["batch_size"])
test_loader = torch.utils.data.DataLoader(test_pytorch_dataset,
                                          batch_size=config["batch_size"])

dataloaders = {"train" : [train_loader,len(train_pytorch_dataset)],
               "valid" : [valid_loader,len(validation_pytorch_dataset)],
               "test" : [test_loader,len(test_pytorch_dataset)]

}

In [23]:
#Test functionality of Dataloader
demo_loader  = torch.utils.data.DataLoader(train_pytorch_dataset, batch_size=10)
batch = next(iter(demo_loader))
inputs, targets = batch

In [24]:
inputs["input_ids"].shape

torch.Size([10, 2048])

In [68]:
targets["labels"].shape

torch.Size([10])

In [70]:
train_pytorch_dataset[0][0]["input_ids"][305:315]

tensor([   374,    264,   4363,   1659,     13, 128001, 128001, 128001, 128001,
        128001])

In [58]:
train_pytorch_dataset[0][0]["attention_mask"][305:315]

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

## Test model forward method

In [26]:
demo_loader  = torch.utils.data.DataLoader(train_pytorch_dataset, batch_size=1)
batch = next(iter(demo_loader))
inputs, targets = batch



with torch.no_grad():
    inputs = {k : v.to(device) for k,v in inputs.items()}
    targets = {k : v.to(device) for k,v in targets.items()}


    model.eval()
    outputs = model(**inputs)
outputs.loss

{'logits': tensor([[ 1.2810,  3.6863, -1.7126,  1.1925, -5.4359,  3.5643]],
        device='cuda:0')}

## Creating Llama-3 tokenized dataset from huggingface dataset object

In [48]:
def llama_preprocessing_function(examples):
    return tokenizer(examples['full_text'], truncation=True, max_length=config["max_length"])

cols_to_delete = ["full_text"]

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True,remove_columns=cols_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("score", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/13845 [00:00<?, ? examples/s]

Map:   0%|          | 0/3462 [00:00<?, ? examples/s]

## Data collator

A data collator prepares batches of data for training or inference in machine learning, ensuring uniform formatting and adherence to model input requirements. This is especially crucial for variable-sized inputs like text sequences.

Functions of Data Collator

1.- Padding: Uniformly pads sequences to the length of the longest sequence using a special token, allowing simultaneous batch processing.

2.- Batching: Groups individual data points into batches for efficient processing.

3.- Handling Special Tokens: Adds necessary special tokens to sequences.

4.- Converting to Tensor: Transforms data into tensors, the required format for machine learning frameworks.

In [55]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer,max_length=config["max_length"])

## Evaluation metrics

In [58]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}



## Training Model

In [59]:
import torch.nn.functional as F

In [60]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [67]:
training_args = TrainingArguments(
    output_dir = 'test_hf_trainer',
    learning_rate = 1e-4,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

In [68]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

/tmp/ipykernel_124089/2851958097.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_result = trainer.train()
     